In [14]:
import lmql
import json
import pandas as pd
from pprint import pprint
import nest_asyncio
nest_asyncio.apply()

In [2]:
@lmql.query
def chain_of_thought(question):
    '''lmql
    # Q&A prompt template
    "Q: {question}\n"
    "A: Let's think step by step.\n"
    "[REASONING]"
    "Thus, the answer is:[ANSWER]."

    # return just the ANSWER to the caller
    return ANSWER
    '''

print(chain_of_thought('Today is the 12th of June, what day was it 1 week ago?'))


 5th of June.


In [3]:
@lmql.query
def chain_of_thought(review):
    '''lmql
    argmax
        
        """Review: {review}
        Q: What is the underlying sentiment of this review and why?
        A:[ANALYSIS]""" where not "\n" in ANALYSIS
        
        "Based on this, the overall sentiment of the message can be considered to be[CLS]" where CLS in [" positive", " neutral", " negative"]
        
        if CLS == " positive":
            "What is it that they liked about their stay? [FURTHER_ANALYSIS]"
        elif CLS == " neutral":
            "What is it that could have been improved? [FURTHER_ANALYSIS]"
        elif CLS == " negative":
            "What is it that they did not like about their stay? [FURTHER_ANALYSIS]"
    where
    STOPS_AT(FURTHER_ANALYSIS, ".")
    '''

review = """We had a great stay. Hiking in the mountains 
                    was fabulous and the food is really good."""

pprint(chain_of_thought(review))


/opt/homebrew/anaconda3/envs/gen-ai-and-llm-integration/lib/python3.10/site-packages/lmql/runtime/bopenai/batched_openai.py:752: OpenAILogitBiasLimitationWarning: the required logit_bias is too large to be handled by the OpenAI API and will be limited to the first 300 tokens. This can lead to the violation of the provided constraints or undesired model output. To avoid this use less broad or no constraints.
  warnings.warn("the required logit_bias is too large to be handled by the OpenAI API and will be limited to the first 300 tokens. This can lead to the violation of the provided constraints or undesired model output. To avoid this use less broad or no constraints.", category=OpenAILogitBiasLimitationWarning)
/opt/homebrew/anaconda3/envs/gen-ai-and-llm-integration/lib/python3.10/site-packages/lmql/runtime/bopenai/batched_openai.py:507: OpenAIAPIWarning: OpenAI API: Underlying stream of OpenAI complete() call failed with error

 (<class 'TimeoutError'>)

Retrying... (attempt: 0)
  war

LMQLResult(prompt='Review: We had a great stay. Hiking in the mountains \n'
                  '                    was fabulous and the food is really '
                  'good.\n'
                  'Q: What is the underlying sentiment of this review and '
                  'why?\n'
                  'A: The underlying sentiment of this review is positive '
                  'because the reviewer had a great stay, enjoyed hiking in '
                  'the mountains, and found the food to be really good.Based '
                  'on this, the overall sentiment of the message can be '
                  'considered to be positiveWhat is it that they liked about '
                  'their stay? \n'
                  'The reviewer liked the hiking in the mountains and the '
                  'food.',
           variables={'ANALYSIS': ' The underlying sentiment of this review is '
                                  'positive because the reviewer had a great '
                                

In [8]:
@lmql.query
def extract_product_information(text):
    '''lmql
    # Task prompt template
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    "**Task**: Extract product information from a given text.\n\n"
    "**Input**:\n"
    "{text}\n\n"
    "**Instructions**:\n"
    "1. Extract the unique product key from the input text, which is typically a 6/7 digit number before the colon mark.\n"
    "2. Identify the product type with considering the state of matter (e.g., shampoo, soap, hand wash, shower gel, laundry capsules, etc.) that is mentioned in the textual description.\n"
    "3. If the product type cannot be identified or is unknown, return 'N/A'.\n"
    "4. Output the answer in JSON format, with the product key and product type as key-value pairs.\n"
    "**Output Format**:\n"
    "{{\"product_key\": \"product_type\"}}\n\n"
    "Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    
    "[ANSWER]"
    
    # return only the extracted JSON
    return ANSWER
    '''

In [33]:
txt_input = """
["284367: L'Oreal Paris Matte Signature Eyeliner 11ml. Black <p>Matt signature eyeliner will help to sharpen your eyeliner girls. Not to be difficult and time consuming anymore. The special design allows long handle grip. Formula ecstasy and Matt Black paint recipe sharp flick the stick waterproof long-lasting all day.</p><p><img src=https://www.watsons.co.th/medias/New-Project.jpg?context=bWFzdGVyfHd0Y3RoL2ltYWdlc3w0OTg3OTJ8aW1hZ2UvanBlZ3xoZTAvaGZhLzExNjkxNTUzMDMwMTc0L05ldyBQcm9qZWN0LmpwZ3xkMDQ4YmJhZGNjMjBlZDE4MzUxOTUwYzJmZTJhYjljZjkyZjBmOGRjMTBiNTVjY2M3OTQ4NzA0NjE4NGY2NTU4 /></p>", '283906: Catrice Calligraph Pro Precise 24h Matt Liner Waterproof 010 1.2 ml 97 percent agree: Intense black strong coverage and high precision application. This waterproof eyeliner is an all-rounder. Accurate lines? No problem thanks to the ultra-fine tip. The pen glides over the eyelid the line is perfectly in place - for up to 24 hours. The extremely intensive colour payoff is also visible over eyeshadow and make-up.', '284839: Browit Smooth And Slim Inner Eyeliner 0.1g. Deep Brown This pressed cream eyeliner provides smooth and precise lines with its slim tip of only 2 mm and pigment-rich color as well as being waterproof and sweat-proof. In addition this item also comes with an automatic twist-up design to facilitate the application that is specially created by Nongchat a renowned and professional makeup artist in Thailand.', '295492: In2it Slim Perfect Gel Liner 0.15g. 02 Brunette In2it Slim Perfect Gel Liner 0.15g. Eyeliner Pencil Bestselling Eyeliner Pencil Eyeliner pencils are good gel liners write eyeliner. Gel line sells well gel liner is good in liner in liner sells well in liner is good the soil is written the eyeliner is waterproof. Eyeliner is not messy waterproof gel liner waterproof eyeliner no gel liner no mess no panda no panda eyeliner. Brown Eyeliner Pencil Brown gel liner Beautiful lines crisp easy to write easy to carry gel liner eyeliner waterproof waterproof gel liner.', '275447: Canmake Creamy Touch Eye Liner 0.08g.01 Deep Black You’ll adore the tantalizingly smooth texture!1.5mm hyper-slim gel eyelinerWaterproofResistant to water sweat tears and sebumContains beautifying ingredients']
"""

txt_input = """
["284367: L'Oreal Paris Matte Signature Eyeliner 11ml. Black <p>Matt signature eyeliner will help to sharpen your eyeliner girls. Not to be difficult and time consuming anymore. The special design allows long handle grip. Formula ecstasy and Matt Black paint recipe sharp flick the stick waterproof long-lasting all day.</p><p><img src=https://www.watsons.co.th/medias/New-Project.jpg?context=bWFzdGVyfHd0Y3RoL2ltYWdlc3w0OTg3OTJ8aW1hZ2UvanBlZ3xoZTAvaGZhLzExNjkxNTUzMDMwMTc0L05ldyBQcm9qZWN0LmpwZ3xkMDQ4YmJhZGNjMjBlZDE4MzUxOTUwYzJmZTJhYjljZjkyZjBmOGRjMTBiNTVjY2M3OTQ4NzA0NjE4NGY2NTU4 /></p>"]
"""

result_cleaned = extract_product_information(txt_input)
result_cleaned = json.loads(result_cleaned)
result_cleaned

{'product_key': '284367', 'product_type': 'eyeliner'}

In [34]:
# Assuming result_cleaned is the response
if "product_key" in result_cleaned and "product_type" in result_cleaned:
    # Case where the result contains only one product_key and product_type
    pd_data = pd.DataFrame([result_cleaned])
else:
    # Case where the result contains multiple product_key and product_type pairs
    pd_data = pd.DataFrame.from_dict(result_cleaned, orient="index").reset_index()
    pd_data.columns = ["product_key", "product_type"]

pd_data    

,product_key,product_type
0,284367,eyeliner
